## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams, bigrams, trigrams
from pymystem3 import Mystem
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from ngram import NGram
import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/vlad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/vlad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vlad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
%%capture
!wget https://github.com/sismetanin/rureviews/blob/master/women-clothing-accessories.3-class.balanced.csv\?raw\=true

In [ ]:
df = pd.read_csv("women-clothing-accessories.3-class.balanced.csv?raw=true", sep="\t")
df.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


### Токенизация

In [ ]:
df['tokenized'] = df['review'].transform(word_tokenize)
df.head()

,review,sentiment,tokenized
0,качество плохое пошив ужасный (горловина напер...,negative,"[качество, плохое, пошив, ужасный, (, горловин..."
1,"Товар отдали другому человеку, я не получила п...",negative,"[Товар, отдали, другому, человеку, ,, я, не, п..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"[Ужасная, синтетика, !, Тонкая, ,, ничего, общ..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"[товар, не, пришел, ,, продавец, продлил, защи..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"[Кофточка, голая, синтетика, ,, носить, не, во..."


### Приведение к нижнему регистру


In [ ]:
df['tokenized'] = df['tokenized'].transform(lambda t: [word.lower() for word in t])
df.head()

,review,sentiment,tokenized
0,качество плохое пошив ужасный (горловина напер...,negative,"[качество, плохое, пошив, ужасный, (, горловин..."
1,"Товар отдали другому человеку, я не получила п...",negative,"[товар, отдали, другому, человеку, ,, я, не, п..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"[ужасная, синтетика, !, тонкая, ,, ничего, общ..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"[товар, не, пришел, ,, продавец, продлил, защи..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"[кофточка, голая, синтетика, ,, носить, не, во..."


### Удаление стоп-слов

In [ ]:
sw = set(stopwords.words('russian'))
df['tokenized'] = df['tokenized'].transform(lambda t: [word for word in t if word not in sw])
df.head()

,review,sentiment,tokenized
0,качество плохое пошив ужасный (горловина напер...,negative,"[качество, плохое, пошив, ужасный, (, горловин..."
1,"Товар отдали другому человеку, я не получила п...",negative,"[товар, отдали, другому, человеку, ,, получила..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"[ужасная, синтетика, !, тонкая, ,, общего, пре..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"[товар, пришел, ,, продавец, продлил, защиту, ..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"[кофточка, голая, синтетика, ,, носить, возмож..."


### Лемматизация


In [ ]:
m = Mystem()
df['tokenized'] = df['tokenized'].transform(lambda t: ''.join(m.lemmatize(' '.join(t))))
df.head()

,review,sentiment,tokenized
0,качество плохое пошив ужасный (горловина напер...,negative,качество плохой пошив ужасный ( горловина напе...
1,"Товар отдали другому человеку, я не получила п...",negative,"товар отдавать другой человек , получать посыл..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"ужасный синтетика ! тонкий , общий представлят..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"товар приходить , продавец продлять защита мой..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"кофточка голый синтетика , носить возможно .\n"


### Векторизация и построение модели

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.tokenized, df.sentiment, train_size = 0.5)

##### Ngram

In [ ]:
def ngram(n_range, analyzer='word'):
    count_vectorizer = CountVectorizer(ngram_range=n_range, analyzer=analyzer)
    count_x_train = count_vectorizer.fit_transform(x_train)

    count_mnb = MultinomialNB()
    count_mnb.fit(count_x_train, y_train)

    count_vectorizer_test = count_vectorizer.transform(x_test)

    count_prediction = count_mnb.predict(count_vectorizer_test)
    
    return classification_report(y_test, count_prediction, output_dict=True)

##### Tf-idf

In [ ]:
def tfidf(n_range, min_df, max_df, max_features):
    tfidf_vectorizer = TfidfVectorizer(
        ngram_range=n_range,
        max_df=max_df,
        min_df=min_df,
        max_features=max_features)
    tfidf_x_train = tfidf_vectorizer.fit_transform(x_train)

    tfidf_mnb = MultinomialNB()
    tfidf_mnb.fit(tfidf_x_train, y_train)

    tfidf_vectorizer_test = tfidf_vectorizer.transform(x_test)
    tfidf_prediction = tfidf_mnb.predict(tfidf_vectorizer_test)
    
    return classification_report(y_test, tfidf_prediction, output_dict=True)

### Оценка качества модели

In [ ]:
ngram_max = 4
ngram_min = 1
min_df_range = [0, 0.01]
max_df_range = [0.2, 0.5] #[0.2, 0.25, 0.5]
features_range = [10_000, 30_000]

all_results = []

def calculate_res_count():
    for analyzer in ['word', 'char']:
        for i in range(ngram_min, ngram_max):
            for j in range(i, ngram_max):
                all_results.append([ngram((i, j), analyzer),
                                   f'Ngrams: ({i},{j})',
                                   analyzer])

def calculate_res_tfidf():
    for i in range(ngram_min, ngram_max):
        print(i)
        for j in range(i, ngram_max):
            for max_d in max_df_range:
                for min_d in min_df_range:
                    for features_num in features_range:
                        all_results.append([tfidf((i, j), min_d, max_d, features_num),
                                           f'df\'s: [{min_d},{max_d}], Features: {features_num}, Ngrams: ({i},{j})',
                                           'tfidf'])

calculate_res_count()
print('50% Done')
calculate_res_tfidf()

50% Done
1
2
3


In [ ]:
for_df = []

for res in all_results:
    precision = res[0]['weighted avg']['precision']
    recall = res[0]['weighted avg']['recall']
    f1 = res[0]['weighted avg']['f1-score']
    
    params = res[1]
    vect = res[2]
    
    for_df.append({'Precision':precision, 
                   'Recall':recall, 
                   'f1-score':f1,
                   'Vectorizer': vect,
                   'Params':params})

final_df = pd.DataFrame(for_df)

In [ ]:
final_df.sort_values(ascending=False, by='Precision')

,Precision,Recall,f1-score,Vectorizer,Params
25,0.724491,0.716644,0.718112,tfidf,"df's: [0,0.5], Features: 30000, Ngrams: (1,2)"
33,0.724096,0.714600,0.716289,tfidf,"df's: [0,0.5], Features: 30000, Ngrams: (1,3)"
21,0.723564,0.715689,0.717065,tfidf,"df's: [0,0.2], Features: 30000, Ngrams: (1,2)"
29,0.723354,0.714067,0.715640,tfidf,"df's: [0,0.2], Features: 30000, Ngrams: (1,3)"
24,0.723171,0.712978,0.714850,tfidf,"df's: [0,0.5], Features: 10000, Ngrams: (1,2)"
32,0.722553,0.711156,0.713110,tfidf,"df's: [0,0.5], Features: 10000, Ngrams: (1,3)"
20,0.721702,0.711511,0.713319,tfidf,"df's: [0,0.2], Features: 10000, Ngrams: (1,2)"
28,0.721531,0.710222,0.712076,tfidf,"df's: [0,0.2], Features: 10000, Ngrams: (1,3)"
1,0.718697,0.716533,0.716727,word,"Ngrams: (1,2)"
2,0.716710,0.716489,0.716176,word,"Ngrams: (1,3)"


In [ ]:
print(final_df.to_string())

    Precision    Recall  f1-score Vectorizer                                            Params
0    0.709495  0.703244  0.703986       word                                     Ngrams: (1,1)
1    0.718697  0.716533  0.716727       word                                     Ngrams: (1,2)
2    0.716710  0.716489  0.716176       word                                     Ngrams: (1,3)
3    0.687900  0.688156  0.687696       word                                     Ngrams: (2,2)
4    0.681076  0.683089  0.681767       word                                     Ngrams: (2,3)
5    0.612147  0.589133  0.594417       word                                     Ngrams: (3,3)
6    0.563886  0.534889  0.536473       char                                     Ngrams: (1,1)
7    0.658019  0.637044  0.641067       char                                     Ngrams: (1,2)
8    0.694208  0.676911  0.680443       char                                     Ngrams: (1,3)
9    0.664710  0.646733  0.650506       char      

### Выводы и обьяснения

Так как датасет представляет собой набор отзывов о товарах, где каждый отзыв является набором из нескольких коротких предложений с частовстречающейся лингвистичекой конструкцией прилагательное-существительное или существительное-прилагательное (например: *товар бракованный*, *плохой продавец*, *долгая доставка*), то большая верхняя граница Ngram скорее усугубит обучение.

Как видно из таблицы параметр Features крайне слабо сказывается на результатах, а в большинстве случаев вовсе никак не влияет на них.

Возможных значений параметра min_df всего 2 так как при увеличении даже на 0.01 результаты предсказания падают значительно.

Увеличение параметра max_df в свою очередь улучшает результаты. По видимому это происходит из-за того, что во многих отзывах часто встречаются такие слова как *товар*, *качество*, *доставка*, которые сами по себе не несут отрицательного или положительного контекста и гораздо важнее какой эпитет с ними связан.


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
t1 = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit'
re.findall(r"\b\w{1,2}", t1)

['Lo', 'ip', 'do', 'si', 'am', 'co', 'ad', 'el']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
t2 = ' sed do eiusmod. tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam.'

re.split(r'\.', t2, maxsplit=2)

[' sed do eiusmod',
 ' tempor incididunt ut labore et dolore magna aliqua',
 ' Ut enim ad minim veniam.']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
t3 = ' quis1 nostrud exerc4itation3 ul0lamco laboris nis7i ut 321'
re.sub(r'\d', 'DIG', t3)

' quisDIG nostrud exercDIGitationDIG ulDIGlamco laboris nisDIGi ut DIGDIGDIG'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
t4 = '  aliquip ex ea commodo consequat. https://abc.bla.bla.shouldiblamecaching.com/a/asdnv/  Duis aute irure dolor '
re.sub(r'http[s]?://[\w.]+[/\w]+/?', '', t4)

'  aliquip ex ea commodo consequat.   Duis aute irure dolor '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
t5 = 'in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
re.compile(r'\w{4,}').findall(t5)

['reprehenderit',
 'voluptate',
 'velit',
 'esse',
 'cillum',
 'dolore',
 'fugiat',
 'nulla',
 'pariatur',
 'Excepteur',
 'sint',
 'occaecat',
 'cupidatat',
 'proident',
 'sunt',
 'culpa',
 'officia',
 'deserunt',
 'mollit',
 'anim',
 'laborum']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
t6 = 'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz'
re.compile(r'@[\w.]+').findall(t6)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']